In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
from kaggle import api
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
!kaggle competitions download -c rohlik-orders-forecasting-challenge

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 65.8MB/s]


In [ ]:
import zipfile
z=zipfile.ZipFile('/content/rohlik-orders-forecasting-challenge.zip','r')
z.extractall('/content')
z.close()

Data exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
df_train.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [ ]:
df_train[df_train.columns]

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,...,month,year,day_of_year,is_weekend,day_of_week_sin,day_of_week_cos,month_sin,month_cos,place_name,date_from_id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,...,12,2020,340,1,-0.974928,-0.222521,-2.449294e-16,1.000000e+00,Prague,2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,...,12,2020,341,1,-0.781831,0.623490,-2.449294e-16,1.000000e+00,Prague,2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,...,12,2020,342,0,0.000000,1.000000,-2.449294e-16,1.000000e+00,Prague,2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,...,12,2020,343,0,0.781831,0.623490,-2.449294e-16,1.000000e+00,Prague,2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,...,12,2020,344,0,0.974928,-0.222521,-2.449294e-16,1.000000e+00,Prague,2020-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,6733.0,NaN,0,0,0,0,0,0,...,3,2024,70,1,-0.781831,0.623490,1.000000e+00,6.123234e-17,Budapest,2024-03-10
7336,Budapest_1,2024-03-11,6492.0,NaN,0,0,0,0,0,0,...,3,2024,71,0,0.000000,1.000000,1.000000e+00,6.123234e-17,Budapest,2024-03-11
7337,Budapest_1,2024-03-12,6661.0,NaN,0,0,0,0,0,0,...,3,2024,72,0,0.781831,0.623490,1.000000e+00,6.123234e-17,Budapest,2024-03-12
7338,Budapest_1,2024-03-13,6843.0,NaN,0,0,0,0,0,0,...,3,2024,73,0,0.974928,-0.222521,1.000000e+00,6.123234e-17,Budapest,2024-03-13


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   warehouse               7340 non-null   object 
 1   date                    7340 non-null   object 
 2   orders                  7340 non-null   float64
 3   holiday_name            218 non-null    object 
 4   holiday                 7340 non-null   int64  
 5   shutdown                7340 non-null   int64  
 6   mini_shutdown           7340 non-null   int64  
 7   shops_closed            7340 non-null   int64  
 8   winter_school_holidays  7340 non-null   int64  
 9   school_holidays         7340 non-null   int64  
 10  blackout                7340 non-null   int64  
 11  mov_change              7340 non-null   float64
 12  frankfurt_shutdown      7340 non-null   int64  
 13  precipitation           7070 non-null   float64
 14  snow                    7070 non-null   

In [ ]:
df_train['id'].unique()

array(['Prague_1_2020-12-05', 'Prague_1_2020-12-06',
       'Prague_1_2020-12-07', ..., 'Budapest_1_2024-03-12',
       'Budapest_1_2024-03-13', 'Budapest_1_2024-03-14'], dtype=object)

In [ ]:
df_test.head()

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   warehouse               397 non-null    object
 1   date                    397 non-null    object
 2   holiday_name            17 non-null     object
 3   holiday                 397 non-null    int64 
 4   shops_closed            397 non-null    int64 
 5   winter_school_holidays  397 non-null    int64 
 6   school_holidays         397 non-null    int64 
 7   id                      397 non-null    object
dtypes: int64(4), object(4)
memory usage: 24.9+ KB


Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

def preprocess_data(df_train, df_test, unique_threshold=3, polynomial_degree=2, treat_numerical_as_categorical=False):
    # Convert 'date' columns to datetime
    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    # Extract date features
    for df in [df_train, df_test]:
        df['day_of_week'] = df['date'].dt.dayofweek
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['day_of_year'] = df['date'].dt.dayofyear
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # 5 = Saturday, 6 = Sunday

        # Cyclical encoding for month and day_of_week
        df['day_of_week_sin'] = np.sin(df['day_of_week'] * (2. * np.pi / 7))
        df['day_of_week_cos'] = np.cos(df['day_of_week'] * (2. * np.pi / 7))
        df['month_sin'] = np.sin(df['month'] * (2. * np.pi / 12))
        df['month_cos'] = np.cos(df['month'] * (2. * np.pi / 12))

    # Extract place names and dates from 'id'
    for df in [df_train, df_test]:
        df['place_name'] = df['id'].str.split('_').str[0]
        df['date_from_id'] = df['id'].str.split('_').str[2]
        df['date_from_id'] = pd.to_datetime(df['date_from_id'])

    # Drop the 'date' and 'id' columns
    df_train = df_train.drop(columns=['date', 'id'])
    df_test = df_test.drop(columns=['date', 'id'])

    # Handle missing values in 'holiday_name'
    df_train['holiday_name'] = df_train['holiday_name'].fillna('unknown')
    df_test['holiday_name'] = df_test['holiday_name'].fillna('unknown')

    # Separate 'orders' column for y_train
    if 'orders' not in df_train.columns:
        raise KeyError("Column 'orders' is not found in df_train")

    y_train = df_train['orders']
    X_train = df_train.drop(columns=['orders'])

    # Ensure common columns between train and test
    common_columns = [col for col in X_train.columns if col in df_test.columns]
    X_train = X_train[common_columns]
    df_test = df_test[common_columns]

    # Identify categorical columns (excluding cyclical encoding features)
    categorical_columns = ['warehouse', 'holiday_name', 'place_name', 'is_weekend']

    # Determine which numerical columns should be treated as categorical
    numerical_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
    numerical_non_cyclical = [col for col in numerical_columns if col not in ['day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos']]

    if treat_numerical_as_categorical:
        # Treat all numerical columns except cyclical ones as categorical
        categorical_columns += numerical_non_cyclical
        numerical_columns = ['day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos']
    else:
        # Use the unique threshold to determine which numerical columns to treat as categorical
        categorical_numerical_columns = [col for col in numerical_non_cyclical
                                          if X_train[col].nunique() <= unique_threshold]

        # Add categorical numerical columns to the categorical columns list
        categorical_columns += categorical_numerical_columns

        # Remaining numerical columns are not categorical
        numerical_columns = [col for col in numerical_non_cyclical
                              if col not in categorical_numerical_columns]
        # Ensure cyclical features are included in numerical columns
        numerical_columns += ['day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos']

    print(f'Categorical columns: {categorical_columns}')
    print(f'Numerical columns: {numerical_columns}')

    # Handle missing values and encoding for categorical columns
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Handle missing values, scaling, and polynomial features for numerical columns
    if numerical_columns:
        numerical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=polynomial_degree, include_bias=False))
        ])
        numerical_transformer = ('num', numerical_pipeline, numerical_columns)
    else:
        numerical_transformer = None

    # Combine pipelines
    transformers = []
    if numerical_transformer:
        transformers.append(numerical_transformer)
    transformers.append(('cat', categorical_pipeline, categorical_columns))

    preprocessor = ColumnTransformer(
        transformers=transformers
    )

    # Preprocess training and test data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(df_test)

    return X_train_processed, y_train, X_test_processed



X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test)


Categorical columns: ['warehouse', 'holiday_name', 'place_name', 'is_weekend', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'is_weekend']
Numerical columns: ['day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos']


V2 preprocessing

In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df_train, df_test, unique_threshold=3, polynomial_degree=2, treat_numerical_as_categorical=False):
    # Convert 'date' columns to datetime
    df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')
    df_test['date'] = pd.to_datetime(df_test['date'], errors='coerce')

    # Feature engineering
    for df in [df_train, df_test]:
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['day_of_week'] = df['date'].dt.dayofweek
        df['is_weekend'] = (df['date'].dt.dayofweek >= 5).astype(int)
        df['is_end_of_month'] = df['date'].dt.is_month_end.astype(int)
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
        df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
        df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    # Handle categorical features
    categorical_features = ['holiday_name', 'warehouse', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'is_weekend', 'is_end_of_month']

    # Explicitly handle 'day' and 'month' as categorical
    categorical_features += ['day', 'month']

    # Apply label encoding for categorical features
    label_encoders = {}
    for feature in categorical_features:
        le = LabelEncoder()
        le.fit(df_train[feature].astype(str))
        df_train[feature] = le.transform(df_train[feature].astype(str))
        df_test[feature] = le.transform(df_test[feature].astype(str))
        label_encoders[feature] = le

    # Drop the 'date' and 'id' columns
    df_train = df_train.drop(columns=['date', 'id'], errors='ignore')
    df_test = df_test.drop(columns=['date', 'id'], errors='ignore')

    # Separate target variable
    if 'orders' not in df_train.columns:
        raise KeyError("Column 'orders' is not found in df_train")

    y_train = df_train['orders']
    X_train = df_train.drop(columns=['orders'])
    X_test = df_test

    # Ensure common columns between train and test
    common_columns = [col for col in X_train.columns if col in X_test.columns]
    X_train = X_train[common_columns]
    X_test = X_test[common_columns]

    # Identify categorical columns
    categorical_columns = [col for col in categorical_features if col in X_train.columns]

    # Determine numerical columns (excluding explicitly categorical ones like 'day' and 'month')
    numerical_columns = [col for col in X_train.columns if col not in categorical_columns]

    if treat_numerical_as_categorical:
        # Treat all numerical columns as categorical
        categorical_columns += numerical_columns
        numerical_columns = []
    else:
        # Use the unique threshold to determine which numerical columns to treat as categorical
        categorical_numerical_columns = [col for col in numerical_columns
                                          if X_train[col].nunique() <= unique_threshold]
        categorical_columns += categorical_numerical_columns
        numerical_columns = [col for col in numerical_columns
                              if col not in categorical_numerical_columns]

    print(f'Categorical columns: {categorical_columns}')
    print(f'Numerical columns: {numerical_columns}')

    # Handle missing values and encoding for categorical columns
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Handle missing values, scaling, and polynomial features for numerical columns
    if numerical_columns:
        numerical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=polynomial_degree, include_bias=False))
        ])
        numerical_transformer = ('num', numerical_pipeline, numerical_columns)
    else:
        numerical_transformer = None

    # Combine pipelines
    transformers = []
    if numerical_transformer:
        transformers.append(numerical_transformer)
    transformers.append(('cat', categorical_pipeline, categorical_columns))

    preprocessor = ColumnTransformer(
        transformers=transformers
    )

    # Preprocess training and test data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    return X_train_processed, y_train, X_test_processed

X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test, treat_numerical_as_categorical=False)


Categorical columns: ['holiday_name', 'warehouse', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'is_weekend', 'is_end_of_month', 'day', 'month']
Numerical columns: ['year', 'day_of_week', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'day_of_week_sin', 'day_of_week_cos']


V3

In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df_train, df_test, unique_threshold=3, polynomial_degree=2, treat_numerical_as_categorical=False):
    # Convert 'date' columns to datetime
    df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')
    df_test['date'] = pd.to_datetime(df_test['date'], errors='coerce')

    # Feature engineering
    def add_date_features(df):
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['day_of_week'] = df['date'].dt.dayofweek
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        df.drop('date', axis=1, inplace=True)
        return df

    df_train = add_date_features(df_train)
    df_test = add_date_features(df_test)

    # Explicitly handle 'day' and 'month' as categorical
    categorical_features = ['holiday_name', 'warehouse', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'is_weekend', 'is_end_of_month', 'day', 'month']

    # Apply label encoding for categorical features
    label_encoders = {}
    for feature in categorical_features:
        if feature in df_train.columns:
            le = LabelEncoder()
            le.fit(df_train[feature].astype(str))
            df_train[feature] = le.transform(df_train[feature].astype(str))
            df_test[feature] = le.transform(df_test[feature].astype(str))
            label_encoders[feature] = le

    # Drop the 'id' column if it exists
    df_train = df_train.drop(columns=['id'], errors='ignore')
    df_test = df_test.drop(columns=['id'], errors='ignore')

    # Separate target variable
    if 'orders' not in df_train.columns:
        raise KeyError("Column 'orders' is not found in df_train")

    y_train = df_train['orders']
    X_train = df_train.drop(columns=['orders'])
    X_test = df_test

    # Ensure common columns between train and test
    common_columns = [col for col in X_train.columns if col in X_test.columns]
    X_train = X_train[common_columns]
    X_test = X_test[common_columns]

    # Identify categorical columns
    categorical_columns = [col for col in categorical_features if col in X_train.columns]

    # Determine numerical columns (excluding explicitly categorical ones like 'day' and 'month')
    numerical_columns = [col for col in X_train.columns if col not in categorical_columns]

    if treat_numerical_as_categorical:
        # Treat all numerical columns as categorical
        categorical_columns += numerical_columns
        numerical_columns = []
    else:
        # Use the unique threshold to determine which numerical columns to treat as categorical
        categorical_numerical_columns = [col for col in numerical_columns
                                          if X_train[col].nunique() <= unique_threshold]
        categorical_columns += categorical_numerical_columns
        numerical_columns = [col for col in numerical_columns
                              if col not in categorical_numerical_columns]

    print(f'Categorical columns: {categorical_columns}')
    print(f'Numerical columns: {numerical_columns}')

    # Handle missing values and encoding for categorical columns
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Handle missing values, scaling, and polynomial features for numerical columns
    if numerical_columns:
        numerical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=polynomial_degree, include_bias=False))
        ])
        numerical_transformer = ('num', numerical_pipeline, numerical_columns)
    else:
        numerical_transformer = None

    # Combine pipelines
    transformers = []
    if numerical_transformer:
        transformers.append(numerical_transformer)
    transformers.append(('cat', categorical_pipeline, categorical_columns))

    preprocessor = ColumnTransformer(
        transformers=transformers
    )

    # Preprocess training and test data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    return X_train_processed, y_train, X_test_processed

X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test, treat_numerical_as_categorical=False)



Categorical columns: ['holiday_name', 'warehouse', 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'is_weekend', 'day', 'month']
Numerical columns: ['year', 'day_of_week']


V4

In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from math import pi

def preprocess_data(df_train, df_test, unique_threshold=3, polynomial_degree=2, treat_numerical_as_categorical=False):
    # Combine train and test data for preprocessing
    all_df = pd.concat([df_train, df_test], sort=False).reset_index(drop=True)

    # Convert 'date' columns to datetime
    all_df['date'] = pd.to_datetime(all_df['date'], errors='coerce')

    # Feature engineering
    date_start = all_df['date'].min()

    all_df['date_year'] = all_df['date'].dt.year.fillna(-1)
    all_df['date_month'] = all_df['date'].dt.month.fillna(-1)
    all_df['date_day'] = all_df['date'].dt.day.fillna(-1)
    all_df['date_day_of_week'] = all_df['date'].dt.dayofweek.fillna(-1)
    all_df['date_week_of_year'] = all_df['date'].dt.isocalendar().week.fillna(-1)
    all_df['date_num'] = (all_df['date'] - date_start).dt.days.fillna(-1)
    all_df['date_day_of_year'] = all_df['date'].dt.dayofyear.fillna(-1)
    all_df['date_day_of_year'] = np.where(
        (all_df['date_year'] % 4 == 0) & (all_df['date_month'] > 2),
        all_df['date_day_of_year'] - 1,
        all_df['date_day_of_year']
    )
    all_df['date_quarter'] = all_df['date'].dt.quarter.fillna(-1)
    all_df['date_is_month_start'] = all_df['date'].dt.is_month_start.astype(int).fillna(-1)
    all_df['date_is_month_end'] = all_df['date'].dt.is_month_end.astype(int).fillna(-1)
    all_df['date_is_quarter_start'] = all_df['date'].dt.is_quarter_start.astype(int).fillna(-1)
    all_df['date_is_quarter_end'] = all_df['date'].dt.is_quarter_end.astype(int).fillna(-1)

    # Add sine and cosine transformations
    all_df['month_sin'] = np.sin(2 * np.pi * all_df['date_month'] / 12)
    all_df['month_cos'] = np.cos(2 * np.pi * all_df['date_month'] / 12)
    all_df['day_sin'] = np.sin(2 * np.pi * all_df['date_day'] / 31)
    all_df['day_cos'] = np.cos(2 * np.pi * all_df['date_day'] / 31)
    all_df['year_sin'] = np.sin(2 * np.pi * all_df['date_day_of_year'] / 365)
    all_df['year_cos'] = np.cos(2 * np.pi * all_df['date_day_of_year'] / 365)

    # Replace NaN values with 'None' for categorical features
    all_df['holiday_name'].fillna('None', inplace=True)

    # OneHotEncoding for 'holiday_name'
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    holiday_encoded = enc.fit_transform(all_df[['holiday_name']])
    encoded_df = pd.DataFrame(holiday_encoded, columns=enc.get_feature_names_out(['holiday_name']))
    all_df = pd.concat([all_df, encoded_df], axis=1)
    all_df = all_df.drop('holiday_name', axis=1)

    # LabelEncoding for 'warehouse'
    le = LabelEncoder()
    all_df['warehouse'] = le.fit_transform(all_df['warehouse'])

    # Create features for day before and after holidays
    all_df['holiday_before'] = all_df['holiday'].shift(1).fillna(0).astype(int)
    all_df['holiday_after'] = all_df['holiday'].shift(-1).fillna(0).astype(int)

    # Separate train and test data
    train_df = all_df[~all_df['orders'].isnull()]
    test_df = all_df[all_df['orders'].isnull()]

    # Drop the 'date' column and any other columns that are no longer needed
    train_df = train_df.drop(columns=['date'], axis=1)
    test_df = test_df.drop(columns=['date'], axis=1)

    # Separate target variable
    y_train = train_df['orders']
    X_train = train_df.drop(columns=['orders'])
    X_test = test_df

    # Ensure common columns between train and test
    common_columns = [col for col in X_train.columns if col in X_test.columns]
    X_train = X_train[common_columns]
    X_test = X_test[common_columns]

    # Identify categorical and numerical columns
    categorical_features = [col for col in X_train.columns if X_train[col].dtype == 'object']
    numerical_columns = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

    if treat_numerical_as_categorical:
        categorical_features += numerical_columns
        numerical_columns = []
    else:
        categorical_numerical_columns = [col for col in numerical_columns if X_train[col].nunique() <= unique_threshold]
        categorical_features += categorical_numerical_columns
        numerical_columns = [col for col in numerical_columns if col not in categorical_numerical_columns]

    print(f'Categorical columns: {categorical_features}')
    print(f'Numerical columns: {numerical_columns}')

    # Define preprocessing pipelines
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

    if numerical_columns:
        numerical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=polynomial_degree, include_bias=False))
        ])
        numerical_transformer = ('num', numerical_pipeline, numerical_columns)
    else:
        numerical_transformer = None

    # Combine pipelines
    transformers = []
    if numerical_transformer:
        transformers.append(numerical_transformer)
    transformers.append(('cat', categorical_pipeline, categorical_features))

    preprocessor = ColumnTransformer(
        transformers=transformers
    )

    # Preprocess training and test data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    return X_train_processed, y_train, X_test_processed






X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test, treat_numerical_as_categorical=False)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Categorical columns: ['id', 'holiday', 'shutdown', 'mini_shutdown', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'blackout', 'mov_change', 'frankfurt_shutdown', 'date_is_month_start', 'date_is_month_end', 'date_is_quarter_start', 'date_is_quarter_end', 'holiday_name_1848 Revolution Memorial Day (Extra holiday)', 'holiday_name_2nd Christmas Day', "holiday_name_All Saints' Day Holiday", 'holiday_name_Christmas Eve', 'holiday_name_Cyrila a Metodej', 'holiday_name_Day of National Unity', 'holiday_name_Den boje za svobodu a demokracii', 'holiday_name_Den ceske statnosti', 'holiday_name_Den osvobozeni', 'holiday_name_Den vzniku samostatneho ceskoslovenskeho statu', 'holiday_name_Easter Monday', 'holiday_name_Good Friday', 'holiday_name_Independent Hungary Day', 'holiday_name_International womens day', 'holiday_name_Jan Hus', 'holiday_name_Labour Day', 'holiday_name_Memorial Day for the Martyrs of Arad', 'holiday_name_Memorial Day for the Victims of the Communist Dictatorships

V5

In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def preprocess_data(df_train, df_test):
    # Drop 'id' from test data
    base_features = df_test.drop(columns=['id']).columns

    # Ensure df_train and df_test have the same columns
    train_df = pd.concat([df_train[base_features], df_train['orders']], axis=1)
    test_df = df_test[base_features]

    # Combine train and test data for preprocessing
    all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

    # Convert the 'date' column to datetime format
    date_start = pd.to_datetime(all_df['date'], errors='coerce').min()
    date_col = pd.to_datetime(all_df['date'], errors='coerce')

    # Extract date features
    all_df['date_year'] = date_col.dt.year.fillna(-1)
    all_df['date_month'] = date_col.dt.month.fillna(-1)
    all_df['date_day'] = date_col.dt.day.fillna(-1)
    all_df['date_day_of_week'] = date_col.dt.dayofweek.fillna(-1)
    all_df['date_week_of_year'] = date_col.dt.isocalendar().week.fillna(-1)
    all_df['date_num'] = (date_col - date_start).dt.days.fillna(-1)
    all_df['date_day_of_year'] = date_col.dt.dayofyear.fillna(-1)
    all_df['date_day_of_year'] = np.where(
        (all_df['date_year'] % 4 == 0) & (all_df['date_month'] > 2),
        all_df['date_day_of_year'] - 1,
        all_df['date_day_of_year']
    )
    all_df['date_quarter'] = date_col.dt.quarter.fillna(-1)
    all_df['date_is_month_start'] = date_col.dt.is_month_start.astype(int).fillna(-1)
    all_df['date_is_month_end'] = date_col.dt.is_month_end.astype(int).fillna(-1)
    all_df['date_is_quarter_start'] = date_col.dt.is_quarter_start.astype(int).fillna(-1)
    all_df['date_is_quarter_end'] = date_col.dt.is_quarter_end.astype(int).fillna(-1)

    # Apply sine and cosine transformations
    all_df['month_sin'] = np.sin(2 * np.pi * all_df['date_month'] / 12)
    all_df['month_cos'] = np.cos(2 * np.pi * all_df['date_month'] / 12)
    all_df['day_sin'] = np.sin(2 * np.pi * all_df['date_day'] / 31)
    all_df['day_cos'] = np.cos(2 * np.pi * all_df['date_day'] / 31)
    all_df['year_sin'] = np.sin(2 * np.pi * all_df['date_day_of_year'] / 365)
    all_df['year_cos'] = np.cos(2 * np.pi * all_df['date_day_of_year'] / 365)

    # Replace NaN values in 'holiday_name' with 'None'
    all_df['holiday_name'].fillna('None', inplace=True)

    # One-Hot Encoding for 'holiday_name'
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    holiday_encoded = enc.fit_transform(all_df[['holiday_name']])
    encoded_df = pd.DataFrame(holiday_encoded, columns=enc.get_feature_names_out(['holiday_name']))
    all_df = pd.concat([all_df, encoded_df], axis=1)
    all_df = all_df.drop('holiday_name', axis=1)

    # Label Encoding for 'warehouse'
    le = LabelEncoder()
    all_df['warehouse'] = le.fit_transform(all_df['warehouse'])

    # Features for the day before and after holidays
    all_df['holiday_before'] = all_df['holiday'].shift(1).fillna(0).astype(int)
    all_df['holiday_after'] = all_df['holiday'].shift(-1).fillna(0).astype(int)

    # Separate train and test data
    train_df_le = all_df[~all_df['orders'].isnull()]
    test_df_le = all_df[all_df['orders'].isnull()]

    # Drop the 'date' column as it is no longer needed
    train_df_le = train_df_le.drop(columns=['date'], axis=1)
    test_df_le = test_df_le.drop(columns=['date'], axis=1)

    # Ensure that the test data does not have 'orders' column
    if 'orders' in test_df_le.columns:
        test_df_le = test_df_le.drop(columns=['orders'])

    # Separate target variable and features
    y_train = train_df_le['orders']
    X_train = train_df_le.drop(columns=['orders'])
    X_test = test_df_le

    return X_train, y_train, X_test

X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


V6

In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def preprocess_data(df_train, df_test):
    # Remove 'id' column if present
    df_train = df_train.drop(columns=['id'], errors='ignore')
    df_test = df_test.drop(columns=['id'], errors='ignore')

    # Base features for the test set
    base_features = df_test.drop(columns=['orders'], errors='ignore').columns
    test_id = df_test['id'] if 'id' in df_test else None

    # Prepare train and test data
    train_df = pd.concat([df_train[base_features], df_train['orders']], axis=1)
    test_df = df_test[base_features]

    # Combine train and test data for preprocessing
    all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

    # Convert the 'date' column to datetime and extract features
    all_df['date'] = pd.to_datetime(all_df['date'], errors='coerce')
    date_start = all_df['date'].min()

    all_df['date_year'] = all_df['date'].dt.year.fillna(-1)
    all_df['date_month'] = all_df['date'].dt.month.fillna(-1)
    all_df['date_day'] = all_df['date'].dt.day.fillna(-1)
    all_df['date_day_of_week'] = all_df['date'].dt.dayofweek.fillna(-1)
    all_df['date_week_of_year'] = all_df['date'].dt.isocalendar().week.fillna(-1)
    all_df['date_num'] = (all_df['date'] - date_start).dt.days.fillna(-1)
    all_df['date_day_of_year'] = all_df['date'].dt.dayofyear.fillna(-1)
    all_df['date_day_of_year'] = np.where(
        (all_df['date_year'] % 4 == 0) & (all_df['date_month'] > 2),
        all_df['date_day_of_year'] - 1,
        all_df['date_day_of_year']
    )
    all_df['date_quarter'] = all_df['date'].dt.quarter.fillna(-1)
    all_df['date_is_month_start'] = all_df['date'].dt.is_month_start.astype(int).fillna(-1)
    all_df['date_is_month_end'] = all_df['date'].dt.is_month_end.astype(int).fillna(-1)
    all_df['date_is_quarter_start'] = all_df['date'].dt.is_quarter_start.astype(int).fillna(-1)
    all_df['date_is_quarter_end'] = all_df['date'].dt.is_quarter_end.astype(int).fillna(-1)

    # Drop original date column
    all_df = all_df.drop(columns=['date'], errors='ignore')

    # Apply sine and cosine transformations
    all_df['month_sin'] = np.sin(2 * np.pi * all_df['date_month'] / 12)
    all_df['month_cos'] = np.cos(2 * np.pi * all_df['date_month'] / 12)
    all_df['day_sin'] = np.sin(2 * np.pi * all_df['date_day'] / 31)
    all_df['day_cos'] = np.cos(2 * np.pi * all_df['date_day'] / 31)
    all_df['year_sin'] = np.sin(2 * np.pi * all_df['date_day_of_year'] / 365)
    all_df['year_cos'] = np.cos(2 * np.pi * all_df['date_day_of_year'] / 365)

    # Fill missing values for holiday_name and apply One-Hot Encoding
    all_df['holiday_name'] = all_df['holiday_name'].fillna('None')
    enc = OneHotEncoder(sparse=False)
    holiday_encoded = enc.fit_transform(all_df[['holiday_name']])
    encoded_df = pd.DataFrame(holiday_encoded, columns=enc.get_feature_names_out(['holiday_name']))
    all_df = pd.concat([all_df, encoded_df], axis=1)
    all_df = all_df.drop(columns=['holiday_name'])

    # Label Encode warehouse column
    le = LabelEncoder()
    all_df['warehouse'] = le.fit_transform(all_df['warehouse'])

    # Create lag features
    all_df['holiday_before'] = all_df['holiday'].shift(1).fillna(0).astype(int)
    all_df['holiday_after'] = all_df['holiday'].shift(-1).fillna(0).astype(int)

    # Drop 'date' column and split back into train and test datasets
    train_df_le = all_df[~all_df['orders'].isnull()].drop(columns=['orders'], errors='ignore')
    test_df_le = all_df[all_df['orders'].isnull()].drop(columns=['orders'], errors='ignore')

    y_train = df_train['orders']

    return train_df_le, y_train, test_df_le












X_train_processed, y_train, X_test_processed = preprocess_data(df_train, df_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
X_train_processed

<7340x163 sparse matrix of type '<class 'numpy.float64'>'
	with 147988 stored elements in Compressed Sparse Row format>

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.8 MB/s eta 0:00:00


Model creation

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor

def train_and_evaluate_models(X_train_processed, y_train, X_test_processed, df_test):
    """
    Trains and evaluates several regression models without hyperparameter tuning,
    selects the best one based on MAE, and makes predictions on the test data.

    Parameters:
    - X_train_processed: Processed training features
    - y_train: Target variable for training
    - X_test_processed: Processed test features
    - df_test: Original test dataframe with 'id' column

    Returns:
    - best_model: The model with the lowest MAE
    - test_predictions: Predictions on the test data
    - results: Dictionary of MAE results for all evaluated models
    """

    # Define models
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        # 'Decision Tree': DecisionTreeRegressor(random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'AdaBoost': AdaBoostRegressor(random_state=42),
        'XGBoost': xgb.XGBRegressor(objective='reg:squarederror'),
        # 'SVR': SVR(),
        # 'MLP Regressor': MLPRegressor(max_iter=1000, random_state=42),
        # 'LightGBM': LGBMRegressor(random_state=42),
        # 'CatBoost': CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, random_state=42, verbose=0)
    }

    # Initialize results dictionary
    results = {}
    best_models = {}

    # Train and evaluate models
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train_processed, y_train)
        predictions = model.predict(X_train_processed)
        mae = mean_absolute_error(y_train, predictions)
        results[name] = mae
        best_models[name] = model
        print(f"{name} MAE: {mae}")

    # Select the best model based on MAE
    best_model_name = min(results, key=results.get)
    best_model = best_models[best_model_name]

    print(f"The best model is: {best_model_name}")

    # Train the best model on the entire training data
    best_model.fit(X_train_processed, y_train)

    # Predict on test data
    test_predictions = best_model.predict(X_test_processed)

    # Optionally save predictions to a CSV file
    output_df = pd.DataFrame({
        'id': df_test['id'],
        'orders': test_predictions
    })
    output_df.to_csv('submission.csv', index=False)

    print("Predictions saved.")

    return best_model, test_predictions, results






V2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def train_and_evaluate_models(X_train_processed, y_train, X_test_processed, df_test, n_splits=10, random_seed=42):
    """
    Trains and evaluates several regression models using cross-validation,
    selects the best one based on MAPE, and makes predictions on the test data.

    Parameters:
    - X_train_processed: Processed training features
    - y_train: Target variable for training
    - X_test_processed: Processed test features
    - df_test: Original test dataframe with 'id' column
    - n_splits: Number of splits for cross-validation
    - random_seed: Random seed for reproducibility

    Returns:
    - best_model: The model with the lowest MAPE
    - test_predictions: Predictions on the test data
    - results: Dictionary of MAPE results for all evaluated models
    """

    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

    # Initialize models
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'Decision Tree': DecisionTreeRegressor(random_state=random_seed),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=random_seed),
        'Gradient Boosting': GradientBoostingRegressor(random_state=random_seed),
        'AdaBoost': AdaBoostRegressor(random_state=random_seed),
        'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=random_seed),
        'SVR': SVR(),
        'MLP Regressor': MLPRegressor(max_iter=1000, random_state=random_seed),
        'LightGBM': LGBMRegressor(random_state=random_seed),
        'CatBoost': CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, random_state=random_seed, verbose=0)
    }

    # Initialize results dictionary
    results = {}

    # Train and evaluate models
    for name, model in models.items():
        print(f"Training {name}...")
        mapes = []
        for train_idx, val_idx in kf.split(X_train_processed):
            X_tr, X_val = X_train_processed.iloc[train_idx], X_train_processed.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model.fit(X_tr, y_tr)
            predictions = model.predict(X_val)
            # Ensure no division by zero
            y_val = np.where(y_val == 0, 1e-10, y_val)
            mape = mean_absolute_percentage_error(y_val, predictions)
            mapes.append(mape)

        # Average MAPE for cross-validation
        avg_mape = np.mean(mapes)
        results[name] = avg_mape
        print(f"{name} MAPE: {avg_mape}")

    # Select the best model based on MAPE
    best_model_name = min(results, key=results.get)
    best_model = models[best_model_name]
    print(f"The best model is: {best_model_name}")

    # Train the best model on the entire training data
    best_model.fit(X_train_processed, y_train)

    # Predict on test data
    test_predictions = best_model.predict(X_test_processed)

    # Optionally save predictions to a CSV file
    output_df = pd.DataFrame({
        'id': df_test['id'],
        'orders': test_predictions
    })
    output_df.to_csv('submission.csv', index=False)

    print("Predictions saved.")

    return best_model, test_predictions, results


In [ ]:
best_model, test_predictions, results = train_and_evaluate_models(X_train_processed, y_train, X_test_processed, df_test)

Training Linear Regression...
Linear Regression MAPE: 0.4706924494360357
Training Ridge Regression...
Ridge Regression MAPE: 0.4710054464695288
Training Lasso Regression...
Lasso Regression MAPE: 0.47140752155583654
Training Decision Tree...
Decision Tree MAPE: 0.05317047372316859
Training Random Forest...
Random Forest MAPE: 0.041236768929512224
Training Gradient Boosting...
Gradient Boosting MAPE: 0.07178414212465242
Training AdaBoost...
AdaBoost MAPE: 0.22155058662685884
Training XGBoost...
XGBoost MAPE: 0.03546898211459169
Training SVR...
SVR MAPE: 0.48697168298138127
Training MLP Regressor...
MLP Regressor MAPE: 0.48054730560832304
Training LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins

In [ ]:
s = pd.read_csv(r'/content/submission.csv')
display(s)

,id,orders
0,Prague_1_2024-03-16,10417.9760
1,Prague_1_2024-03-17,10299.9910
2,Prague_1_2024-03-18,9800.2705
3,Prague_1_2024-03-19,9432.4240
4,Prague_1_2024-03-20,9439.0270
...,...,...
392,Budapest_1_2024-05-11,7217.4840
393,Budapest_1_2024-05-12,6919.1590
394,Budapest_1_2024-05-13,6842.0960
395,Budapest_1_2024-05-14,7298.7812
